# [모듈 1.1] CodeCommit 리파지토리 생성

이 노트북은 CodeCommit 리파지토리를 생성하고, codecommit 폴더를 리파지토리에 Push 하는 노트북 입니다.

## 참고 자료
- [Quick guide to start using AWS CodeCommit via AWS CLI](https://dev.to/tiamatt/quick-guide-to-start-using-aws-codecommit-via-aws-cli-1jg5)
- 개발자 가이드
    - [Setup for HTTPS users using Git credentials](https://docs.aws.amazon.com/codecommit/latest/userguide/setting-up-gc.html)
    - [Create an AWS CodeCommit repository](https://docs.aws.amazon.com/codecommit/latest/userguide/how-to-create-repository.html#how-to-create-repository-cli)
    



# 1. 환경 확인

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [ ]:
%%sh 
git --version

In [ ]:
%%sh
aws --version

# 2. 파라미터 로딩

In [ ]:
%store -r code_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)

import json
print (json.dumps(code_pipeline_serving_dict, indent=2))

repository_name = code_pipeline_serving_dict["serving_code_repo_name"]

In [ ]:
source_code = 'codecommit'


# 3.리파지토리 생성

In [ ]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit create-repository --repository-name $repository_name \
--repository-description "My inference repository" 

# 4. 리파지토리 로컬에 클로닝
### 아래를 위의 결과에서 카피해서 대체 해주세요.
#### clone_url =   "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/mlops-ncf-serving-pipeline"


In [ ]:
import boto3

clone_url = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('cloneUrlHttp')
print(clone_url)


In [ ]:
%%sh -s {clone_url} {repository_name}

clone_url=$1
repository_name=$2

rm -rf $repository_name

git clone $clone_url 

# 5. 소스 코드 복사

In [ ]:
%%sh -s {repository_name} {source_code}
repository_name=$1
source_code=$2

cp -r $source_code/* $repository_name
ls $repository_name

# 6. 코드 파일을 리파지토리에 푸시

In [ ]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
echo $PWD
git add .
git commit -m"Add new files"
git push origin master


# 7. 변수 저장

In [ ]:
serving_repository_name = repository_name
%store serving_repository_name

# 8. Code Commit 생성 확인

![serving_code_commit.png](img/serving_code_commit.png)